# 영화리뷰 텍스트 감성분석하기

imdb 영화 리뷰 평점 데이터를 토대로 사용자들의 리뷰 감성을 분류해 보는 실용적인 텍스트 분류 모델을 구현해 본다.

- 들어가며
- 텍스트 감정분석의 유용성
- 텍스트 데이터의 특징
  - 텍스트를 숫자로 표현하는 방법
  - Embedding 레이어의 등장
- 시퀀스 데이터를 다루는 RNN
- 꼭 RNN이어야 할까?
- IMDB 영화리뷰 감성분석
  - IMDB 데이터셋 분석
  - 딥러닝 모델 설계와 훈련
  - Word2Vec의 적용
- 프로젝트 : 네이버 영화리뷰 감성분석 도전하기
- 프로젝트 제출

## 들어가며

자연어 처리에 주로 활용되는 RNN(Recurrent Neural Network)

CNN(Convolutional Neural Network)이 자연어 처리에서 사용

영화리뷰에 대한 감성분석(sentiment analysis) 를 진행

- 텍스트 데이터를 머신러닝 입출력용 수치데이터로 변환하는 과정을 이해
- RNN의 특징을 이해
- 시퀀셜한 데이터를 다루는 방법을 이해
- 1-D CNN으로도 텍스트를 처리
- 영화리뷰 감성 분류 실습

## 텍스트 감정분석의 유용성

텍스트에 담긴 감성을 분석(Sentimental Analysis)하는 방법 즉, 긍정적인지 혹은 부정적인지를 분류(Classification) 

![텍스트에 담긴 미묘한 어감의 차이를 기계가 이해할 수 있게 하려면 어떻게 해야 할까요?](img/00.png)

텍스트 감성분석은 어떤 점에서 유용할까?
***

- 텍스트 데이터만이 가지고 있는 정보적 특성과 가치
- 다른 데이터 분석 업무에 어떤 점에서 도움
- 기존 분석의 기술적 어려움
- 어떤 점에서 유리

[동아비즈니스리뷰 감성분석 활용 사례 기고](https://dbr.donga.com/article/view/1202/article_no/8891/ac/magazine)

- 광범위 텍스트 데이터 수집 가능
  - 실시간 트렌드 반영 가능
- 기계 학습 vs 감성 사전
  - 각각 다른 어휘
  - 긍부정 원인 판단 X
- 정형 데이터화 가능
- word embedding으로 연산과 정확도를 잡음

## 텍스트 데이터의 특징

- 텍스트를 어떻게 숫자 행렬로 표현?
- 입력 데이터의 순서를 인공지능 모델에 어떻게 반영?

### 텍스트를 숫자로 표현하는 방법

텍스트의 중요한 특징은 그 자체로는 기호일 뿐

단어와 그 단어의 의미를 나타내는 벡터를 짝지어 보려고 하는 것

3개의 짧은 문장으로 이루어진 텍스트 데이터를 처리하는 간단한 예제

In [ ]:
# 처리해야 할 문장을 파이썬 리스트에 옮겨 담았습니다.
sentences=['i feel hungry', 'i eat lunch', 'now i feel happy']

# 파이썬 split() 메소드를 이용해 단어 단위로 문장을 쪼개 봅니다.
word_list = 'i feel hungry'.split()
print(word_list)

In [ ]:
index_to_word={}  # 빈 딕셔너리를 만들어서

# 단어들을 하나씩 채워 봅니다. 채우는 순서는 일단 임의로 하였습니다. 그러나 사실 순서는 중요하지 않습니다. 
# <BOS>, <PAD>, <UNK>는 관례적으로 딕셔너리 맨 앞에 넣어줍니다. 
index_to_word[0]='<PAD>'  # 패딩용 단어
index_to_word[1]='<BOS>'  # 문장의 시작지점
index_to_word[2]='<UNK>'  # 사전에 없는(Unknown) 단어
index_to_word[3]='i'
index_to_word[4]='feel'
index_to_word[5]='hungry'
index_to_word[6]='eat'
index_to_word[7]='lunch'
index_to_word[8]='now'
index_to_word[9]='happy'

print(index_to_word)

In [ ]:
word_to_index = {}

for index, word in index_to_word.items():
  word_to_index[word] = index

#word_to_index={word:index for index, word in index_to_word.items()}
print(word_to_index)

In [ ]:
# 문장 1개를 활용할 딕셔너리와 함께 주면, 단어 인덱스 리스트로 변환해 주는 함수를 만들어 봅시다.
# 단, 모든 문장은 <BOS>로 시작하는 것으로 합니다. 
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index['<BOS>']]+[word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in sentence.split()]

print(get_encoded_sentence('i eat lunch', word_to_index))

In [ ]:
# 여러 개의 문장 리스트를 한꺼번에 숫자 텐서로 encode해 주는 함수입니다. 
def get_encoded_sentences(sentences, word_to_index):
    return [get_encoded_sentence(sentence, word_to_index) for sentence in sentences]

# sentences=['i feel hungry', 'i eat lunch', 'now i feel happy'] 가 아래와 같이 변환됩니다. 
encoded_sentences = get_encoded_sentences(sentences, word_to_index)
print(encoded_sentences)

In [ ]:
# 숫자 벡터로 encode된 문장을 원래대로 decode하는 함수입니다. 
def get_decoded_sentence(encoded_sentence, index_to_word):
    return ' '.join(index_to_word[index] if index in index_to_word else '<UNK>' for index in encoded_sentence[1:])  #[1:]를 통해 <BOS>를 제외

print(get_decoded_sentence([1, 3, 4, 5], index_to_word))

In [ ]:
# 여러 개의 숫자 벡터로 encode된 문장을 한꺼번에 원래대로 decode하는 함수입니다. 
def get_decoded_sentences(encoded_sentences, index_to_word):
    return [get_decoded_sentence(encoded_sentence, index_to_word) for encoded_sentence in encoded_sentences]

# encoded_sentences=[[1, 3, 4, 5], [1, 3, 6, 7], [1, 8, 3, 4, 9]] 가 아래와 같이 변환됩니다.
print(get_decoded_sentences(encoded_sentences, index_to_word))

### Embedding 레이어의 등장

## 시퀀스 데이터를 다루는 RNN

## 꼭 RNN이어야 할까?

## IMDB 영화리뷰 감성분석

### IMDB 데이터셋 분석

### 딥러닝 모델 설계와 훈련

### Word2Vec의 적용

## 프로젝트 : 네이버 영화리뷰 감성분석 도전하기

## 프로젝트 제출

# 코드 블록

# 중요

# 링크

# 이미지